# Activity 3.2 - Transfer Learning

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [ ]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [ ]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [ ]:

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [ ]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [ ]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()



Create two datasets
* one with digits below 5
* one with 5 and above

In [ ]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [ ]:


feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [ ]:


classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [ ]:

model = Sequential(feature_layers + classification_layers)

Check the model summary

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

 Train the  model on the digits 5,6,7,8,9

In [ ]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 45s 186ms/step - loss: 1.6081 - accuracy: 0.2031 - val_loss: 1.5989 - val_accuracy: 0.2152
Epoch 2/5
230/230 [==============================] - 46s 198ms/step - loss: 1.5933 - accuracy: 0.2578 - val_loss: 1.5814 - val_accuracy: 0.3894
Epoch 3/5
230/230 [==============================] - 39s 171ms/step - loss: 1.5763 - accuracy: 0.3259 - val_loss: 1.5623 - val_accuracy: 0.5696
Epoch 4/5
230/230 [==============================] - 41s 180ms/step - loss: 1.5583 - accuracy: 0.3967 - val_loss: 1.5408 - val_accuracy: 0.6931
Epoch 5/5
230/230 [==============================] - 40s 172ms/step - loss: 1.5372 - accuracy: 0.4611 - val_loss: 1.5159 - val_accuracy: 0.7435
Training time: 0:03:31.847680
Test score: 1.5158536434173584
Test accuracy: 0.7434684038162231


Freeze only the feature layers

In [ ]:

for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 activation (Activation)     (None, 26, 26, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                        

Train again the model using the 0 to 4 digits

In [ ]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 17s 69ms/step - loss: 1.5915 - accuracy: 0.2524 - val_loss: 1.5730 - val_accuracy: 0.3573
Epoch 2/5
240/240 [==============================] - 15s 63ms/step - loss: 1.5663 - accuracy: 0.3206 - val_loss: 1.5463 - val_accuracy: 0.4485
Epoch 3/5
240/240 [==============================] - 15s 63ms/step - loss: 1.5416 - accuracy: 0.3793 - val_loss: 1.5199 - val_accuracy: 0.5250
Epoch 4/5
240/240 [==============================] - 15s 63ms/step - loss: 1.5169 - accuracy: 0.4338 - val_loss: 1.4936 - val_accuracy: 0.5795
Epoch 5/5
240/240 [==============================] - 14s 58ms/step - loss: 1.4920 - accuracy: 0.4860 - val_loss: 1.4669 - val_accuracy: 0.6357
Training time: 0:01:22.774231
Test score: 1.4669108390808105
Test accuracy: 0.635726809501648


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [ ]:
model_1 = Sequential(feature_layers + classification_layers)

# Train the model on digits less than 5
train_model(model_1,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 18s 70ms/step - loss: 1.4670 - accuracy: 0.5320 - val_loss: 1.4407 - val_accuracy: 0.7050
Epoch 2/5
240/240 [==============================] - 15s 63ms/step - loss: 1.4425 - accuracy: 0.5774 - val_loss: 1.4134 - val_accuracy: 0.7751
Epoch 3/5
240/240 [==============================] - 14s 59ms/step - loss: 1.4157 - accuracy: 0.6202 - val_loss: 1.3853 - val_accuracy: 0.8227
Epoch 4/5
240/240 [==============================] - 14s 58ms/step - loss: 1.3904 - accuracy: 0.6602 - val_loss: 1.3571 - val_accuracy: 0.8474
Epoch 5/5
240/240 [==============================] - 15s 63ms/step - loss: 1.3633 - accuracy: 0.6978 - val_loss: 1.3284 - val_accuracy: 0.8677
Training time: 0:01:16.208986
Test score: 1.3284306526184082
Test accuracy: 0.8676785230636597


In [ ]:
# Freeze the feature layers and fine-tune on digits 5-9
for l in feature_layers:
    l.trainable = False

train_model(model_1,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 16s 65ms/step - loss: 1.4968 - accuracy: 0.4498 - val_loss: 1.4711 - val_accuracy: 0.5820
Epoch 2/5
230/230 [==============================] - 15s 65ms/step - loss: 1.4720 - accuracy: 0.4985 - val_loss: 1.4459 - val_accuracy: 0.6281
Epoch 3/5
230/230 [==============================] - 15s 64ms/step - loss: 1.4487 - accuracy: 0.5373 - val_loss: 1.4209 - val_accuracy: 0.6774
Epoch 4/5
230/230 [==============================] - 15s 64ms/step - loss: 1.4253 - accuracy: 0.5749 - val_loss: 1.3962 - val_accuracy: 0.7145
Epoch 5/5
230/230 [==============================] - 15s 64ms/step - loss: 1.4038 - accuracy: 0.6132 - val_loss: 1.3720 - val_accuracy: 0.7433
Training time: 0:01:14.894134
Test score: 1.3719946146011353
Test accuracy: 0.7432627081871033


#### Conclusion

> In conclusion, model_1 which was trained on digits 0-4, **outperformed** the previous model trained on digits 5-9. The training time was also notably shorter, from 3:31 minutes to just 1:16 minutes. Fine-tuning on the other set of digits resulted in a significant decrease in accuracy, suggesting that the features learned from digits 0-4 were not transferable to digits 5-9, and vice versa.